# సెషన్ 4 – SLM vs LLM పోలిక

ఫౌండ్రీ లోకల్ ద్వారా నడుస్తున్న చిన్న భాషా మోడల్ మరియు పెద్ద మోడల్ మధ్య లేటెన్సీ మరియు నమూనా ప్రతిస్పందన నాణ్యతను పోల్చండి.


## ⚡ త్వరిత ప్రారంభం

**మెమరీ-ఆప్టిమైజ్డ్ సెటప్ (నవీకరించబడింది):**
1. మోడల్స్ ఆటో-సెలెక్ట్ CPU వేరియంట్లు (ఏ హార్డ్‌వేర్‌పై పనిచేస్తుంది)
2. 7B స్థానంలో `qwen2.5-3b` ఉపయోగిస్తుంది (~4GB RAM సేవ్ చేస్తుంది)
3. పోర్ట్ ఆటో-డిటెక్షన్ (మాన్యువల్ కాన్ఫిగరేషన్ అవసరం లేదు)
4. మొత్తం RAM అవసరం: ~8GB సిఫార్సు చేయబడింది (మోడల్స్ + OS)

**టెర్మినల్ సెటప్ (30 సెకన్లు):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

తర్వాత ఈ నోట్‌బుక్‌ను నడపండి! 🚀


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్
టైమింగ్ మరియు చాట్ అభ్యర్థనలకు అవసరమైన కనీస ప్యాకేజీలు (`foundry-local-sdk`, `openai`, `numpy`) ఇన్‌స్టాల్ చేస్తుంది. పునఃప్రారంభించడానికి సురక్షితం.


# సన్నివేశం
ఒక ప్రాతినిధ్యమైన చిన్న భాషా మోడల్ (SLM) ను పెద్ద మోడల్ తో ఒకే ప్రాంప్ట్ పై పోల్చి ట్రేడ్-ఆఫ్స్ ను వివరించండి:
- **లేటెన్సీ తేడా** (గడియారపు సెకన్లు)
- **టోకెన్ వినియోగం** (ఉపలభ్యమైతే) throughput కు ప్రాక్సీగా
- **నమూనా గుణాత్మక అవుట్పుట్** త్వరిత దృష్టికి
- **స్పీడప్ లెక్కింపు** పనితీరు లాభాలను కొలవడానికి

**పరిసర వేరియబుల్స్:**
- `SLM_ALIAS` - చిన్న భాషా మోడల్ (డిఫాల్ట్: phi-4-mini, సుమారు 4GB RAM)
- `LLM_ALIAS` - పెద్ద భాషా మోడల్ (డిఫాల్ట్: qwen2.5-7b, సుమారు 7GB RAM)
- `COMPARE_PROMPT` - పోలిక కోసం పరీక్ష ప్రాంప్ట్
- `COMPARE_RETRIES` - పునఃప్రయత్నాల సంఖ్య (డిఫాల్ట్: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - సర్వీస్ ఎండ్‌పాయింట్‌ను ఓవర్‌రైడ్ చేయండి (సెట్ చేయకపోతే ఆటో-డిటెక్ట్ అవుతుంది)

**ఇది ఎలా పనిచేస్తుంది (అధికారిక SDK నమూనా):**
1. **FoundryLocalManager** Foundry Local సర్వీస్‌ను ప్రారంభించి నిర్వహిస్తుంది
2. సర్వీస్ నడుస్తున్నట్లయితే ఆటో-స్టార్ట్ అవుతుంది (మాన్యువల్ సెటప్ అవసరం లేదు)
3. మోడల్స్ అలియాస్ల నుండి కాంక్రీట్ ID లకు ఆటోమేటిక్‌గా పరిష్కరించబడతాయి
4. హార్డ్‌వేర్-ఆప్టిమైజ్డ్ వేరియంట్లు ఎంచుకోబడతాయి (CUDA, NPU, లేదా CPU)
5. OpenAI-అనుకూల క్లయింట్ చాట్ కంప్లీషన్స్ చేస్తుంది
6. మెట్రిక్స్‌లు సేకరించబడతాయి: లేటెన్సీ, టోకెన్లు, అవుట్పుట్ నాణ్యత
7. ఫలితాలు పోల్చి స్పీడప్ నిష్పత్తి లెక్కించబడుతుంది

ఈ సూక్ష్మ-పోలిక మీ ఉపయోగానికి పెద్ద మోడల్ వైపు మారడం ఎప్పుడు సరైనదో నిర్ణయించడంలో సహాయపడుతుంది.

**SDK సూచన:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- వర్క్‌షాప్ యుటిల్స్: ../samples/workshop_utils.py నుండి అధికారిక నమూనాను ఉపయోగిస్తుంది

**ప్రధాన లాభాలు:**
- ✅ ఆటోమేటిక్ సర్వీస్ కనుగొనడం మరియు ప్రారంభం
- ✅ సర్వీస్ నడవకపోతే ఆటో-స్టార్ట్
- ✅ బిల్ట్-ఇన్ మోడల్ పరిష్కారం మరియు క్యాచింగ్
- ✅ హార్డ్‌వేర్ ఆప్టిమైజేషన్ (CUDA/NPU/CPU)
- ✅ OpenAI SDK అనుకూలత
- ✅ పునఃప్రయత్నాలతో బలమైన లోపాల నిర్వహణ
- ✅ లోకల్ ఇన్ఫరెన్స్ (క్లౌడ్ API అవసరం లేదు)


## 🚨 ముందస్తు అవసరాలు: Foundry Local నడుస్తుండాలి!

**ఈ నోట్‌బుక్‌ను నడిపించే ముందు**, Foundry Local సర్వీస్ సెట్ అప్ అయి ఉండాలి:

### త్వరిత ప్రారంభ ఆదేశాలు (టెర్మినల్‌లో నడపండి):

```bash
# 1. ఫౌండ్రీ లోకల్ సేవను ప్రారంభించండి
foundry service start

# 2. ఈ పోలికలో ఉపయోగించే డిఫాల్ట్ మోడల్స్ (CPU-ఆప్టిమైజ్డ్) ను లోడ్ చేయండి
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. మోడల్స్ లోడ్ అయ్యాయో లేదో నిర్ధారించండి
foundry model ls

# 4. సేవ ఆరోగ్యాన్ని తనిఖీ చేయండి
foundry service status
```

### ప్రత్యామ్నాయ మోడల్స్ (డిఫాల్ట్‌లు అందుబాటులో లేకపోతే):

```bash
# మరింత చిన్న ప్రత్యామ్నాయాలు (స్మృతి చాలా పరిమితమైనప్పుడు)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# లేదా ఈ నోట్‌బుక్‌లో వాతావరణ చరాలు నవీకరించండి:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # లేదా కనిష్ట స్మృతికి qwen2.5-0.5b
```

⚠️ **ఈ దశలను మీరు దాటివేస్తే**, క్రింది నోట్‌బుక్ సెల్స్ నడిపేటప్పుడు `APIConnectionError` కనిపిస్తుంది.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### వివరణ: కోర్ ఇంపోర్ట్స్
మోడల్ సమాచారం పొందడానికి మరియు చాట్ పూర్తి చేయడానికి ఉపయోగించే టైమింగ్ యుటిలిటీలను మరియు Foundry Local / OpenAI క్లయింట్లను తీసుకువస్తుంది.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### వివరణ: అలియాసులు & ప్రాంప్ట్ సెటప్
చిన్న మరియు పెద్ద మోడల్‌ల కోసం వాతావరణం-సర్దుబాటు అలియాసులను నిర్వచిస్తుంది మరియు ఒక పోలిక ప్రాంప్ట్‌ను కూడా. వాతావరణ వేరియబుల్స్‌ను సర్దుబాటు చేసి వివిధ మోడల్ కుటుంబాలు లేదా పనులపై ప్రయోగాలు చేయండి.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 మెమరీ-ఆప్టిమైజ్డ్ కాన్ఫిగరేషన్

**ఈ నోట్‌బుక్ డిఫాల్ట్‌గా మెమరీ-సమర్థవంతమైన మోడల్స్‌ను ఉపయోగిస్తుంది:**
- `phi-4-mini` → సుమారు 4GB RAM (Foundry Local CPU వెర్షన్‌ను ఆటో-సెలెక్ట్ చేస్తుంది)
- `qwen2.5-3b` → సుమారు 3GB RAM (7B స్థానంలో, ఇది సుమారు 7GB+ అవసరం)

**పోర్ట్ ఆటో-డిటెక్షన్:**
- Foundry Local వేర్వేరు పోర్ట్స్‌ను ఉపయోగించవచ్చు (సాధారణంగా 55769 లేదా 59959)
- క్రింది డయాగ్నోస్టిక్ సెల్ సరైన పోర్ట్‌ను ఆటోమేటిక్‌గా గుర్తిస్తుంది
- మాన్యువల్ కాన్ఫిగరేషన్ అవసరం లేదు!

**మీ వద్ద పరిమిత RAM (<8GB) ఉంటే, మరింత చిన్న మోడల్స్‌ను ఉపయోగించండి:**
```python
SLM = 'phi-3.5-mini'      # సుమారు 2జీబి
LLM = 'qwen2.5-0.5b'      # సుమారు 500ఎంబి
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### వివరణ: ఎగ్జిక్యూషన్ హెల్పర్స్ (Foundry SDK నమూనా)
వర్క్‌షాప్ నమూనాల్లో డాక్యుమెంట్ చేయబడిన అధికారిక Foundry Local SDK నమూనాను ఉపయోగిస్తుంది:

**పద్ధతి:**
- **FoundryLocalManager** - Foundry Local సర్వీస్‌ను ప్రారంభించి నిర్వహిస్తుంది
- **ఆటో-డిటెక్షన్** - ఎండ్‌పాయింట్‌ను ఆటోమేటిక్‌గా కనుగొని సర్వీస్ లైఫ్‌సైకిల్‌ను నిర్వహిస్తుంది
- **మోడల్ రిజల్యూషన్** - అలియాస్లను పూర్తి మోడల్ IDలకు పరిష్కరిస్తుంది (ఉదా: phi-4-mini → phi-4-mini-instruct-cpu)
- **హార్డ్‌వేర్ ఆప్టిమైజేషన్** - అందుబాటులో ఉన్న హార్డ్‌వేర్ (CUDA, NPU, లేదా CPU) కోసం ఉత్తమ వేరియంట్‌ను ఎంచుకుంటుంది
- **OpenAI క్లయింట్** - మేనేజర్ యొక్క ఎండ్‌పాయింట్‌తో OpenAI-అనుకూల API యాక్సెస్ కోసం కాన్ఫిగర్ చేయబడింది

**స్థిరత్వ లక్షణాలు:**
- ఎక్స్‌పోనెన్షియల్ బ్యాక్‌ఆఫ్ రీట్రై లాజిక్ (పరిసరాల ద్వారా కాన్ఫిగర్ చేయదగినది)
- సర్వీస్ నడవకపోతే ఆటోమేటిక్ స్టార్ట్‌అప్
- ప్రారంభం తర్వాత కనెక్షన్ ధృవీకరణ
- వివరమైన లోప నివేదికతో సున్నితమైన లోప నిర్వహణ
- మోడల్ క్యాచింగ్ ద్వారా పునరావృత ప్రారంభం నివారించడం

**ఫలిత నిర్మాణం:**
- లేటెన్సీ కొలత (వాల్ క్లాక్ సమయం)
- టోకెన్ వినియోగం ట్రాకింగ్ (అందుబాటులో ఉంటే)
- నమూనా అవుట్పుట్ (ఓదార్పు కోసం సంక్షిప్తం)
- విఫలమైన అభ్యర్థనల కోసం లోప వివరాలు

ఈ నమూనా అధికారిక SDK నమూనాను అనుసరించే workshop_utils మాడ్యూల్‌ను ఉపయోగిస్తుంది.

**SDK సూచన:**
- ప్రధాన రిపో: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### వివరణ: ప్రీ-ఫ్లైట్ స్వీయ-పరీక్ష
రెండు మోడల్స్ కోసం FoundryLocalManager ఉపయోగించి ఒక తేలికపాటి కనెక్టివిటీ తనిఖీ నిర్వహిస్తుంది. ఇది నిర్ధారిస్తుంది:
- సర్వీస్ అందుబాటులో ఉంది
- మోడల్స్ ప్రారంభించవచ్చు
- అలియాసులు వాస్తవ మోడల్ IDలకు పరిష్కరించబడతాయి
- పోలిక నిర్వహించే ముందు కనెక్షన్ స్థిరంగా ఉంది

setup() ఫంక్షన్ workshop_utils నుండి అధికారిక SDK నమూనాను ఉపయోగిస్తుంది.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ త్వరిత పరిష్కారం: నోట్‌బుక్ నుండి Foundry Local ప్రారంభించండి (ఐచ్ఛికం)

పై డయాగ్నోస్టిక్ సర్వీస్ నడుస్తున్నట్లు చూపించకపోతే, మీరు ఇక్కడినుంచి ప్రారంభించడానికి ప్రయత్నించవచ్చు:

**గమనిక:** ఇది విండోస్‌లో ఉత్తమంగా పనిచేస్తుంది. ఇతర ప్లాట్‌ఫారమ్‌లలో, టెర్మినల్ కమాండ్లను ఉపయోగించండి.


### ⚠️ కనెక్షన్ లోపాలను పరిష్కరించడం

మీరు `APIConnectionError` చూస్తున్నట్లయితే, Foundry Local సర్వీస్ నడుస్తుండకపోవచ్చు లేదా మోడల్స్ లోడ్ కాలేదు. ఈ దశలను ప్రయత్నించండి:

**1. సర్వీస్ స్థితిని తనిఖీ చేయండి:**
```bash
# టెర్మినల్‌లో (నోట్‌బుక్‌లో కాదు):
foundry service status
```

**2. సర్వీస్ ప్రారంభించండి (నడుస్తున్నట్లయితే కాదు):**
```bash
foundry service start
```

**3. అవసరమైన మోడల్స్ లోడ్ చేయండి:**
```bash
# సరిపోల్చడానికి అవసరమైన మోడల్స్‌ను లోడ్ చేయండి
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# లేదా ప్రత్యామ్నాయ మోడల్స్‌ను ఉపయోగించండి:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. మోడల్స్ అందుబాటులో ఉన్నాయో నిర్ధారించుకోండి:**
```bash
foundry model ls
```

**సాధారణ సమస్యలు:**
- ❌ సర్వీస్ నడవడం లేదు → `foundry service start` ను నడపండి
- ❌ మోడల్స్ లోడ్ కాలేదు → `foundry model run <model-name>` ను నడపండి
- ❌ పోర్ట్ ఘర్షణలు → మరొక సర్వీస్ ఆ పోర్ట్ ఉపయోగిస్తున్నదో చూడండి
- ❌ ఫైర్వాల్ అడ్డుకుంటోంది → స్థానిక కనెక్షన్లు అనుమతించబడినట్లు నిర్ధారించుకోండి

**త్వరిత పరిష్కారం:** ప్రీ-ఫ్లైట్ తనిఖీకి ముందు దిగువ డయాగ్నోస్టిక్ సెల్ నడపండి.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ ప్రీ-ఫ్లైట్ చెక్: మోడల్ అందుబాటు

ఈ సెల్ పోలికను నడిపించే ముందు రెండు మోడల్స్ కూడా కాన్ఫిగర్ చేసిన ఎండ్‌పాయింట్ వద్ద చేరుకోగలవో లేదో నిర్ధారిస్తుంది.


### వివరణ: రన్ పోలిక & ఫలితాలు సేకరణ
అధికారిక Foundry SDK నమూనాను ఉపయోగించి రెండు అలియాస్లపై తిరుగుతుంది:
1. ప్రతి మోడల్‌ను setup() తో ప్రారంభించండి (FoundryLocalManager ఉపయోగిస్తుంది)
2. OpenAI-అనుకూల API తో ఇన్ఫరెన్స్ నడపండి
3. లేటెన్సీ, టోకెన్లు, మరియు నమూనా అవుట్పుట్‌ను క్యాప్చర్ చేయండి
4. పోలిక విశ్లేషణతో JSON సారాంశాన్ని ఉత్పత్తి చేయండి

ఇది session04/model_compare.py లోని వర్క్‌షాప్ నమూనాలతో అదే నమూనాను అనుసరిస్తుంది.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### ఫలితాలను అర్థం చేసుకోవడం

**ప్రధాన మెట్రిక్స్:**
- **లేటెన్సీ**: తక్కువ ఉండటం మంచిది - వేగవంతమైన స్పందన సమయాన్ని సూచిస్తుంది
- **టోకెన్లు**: ఎక్కువ throughput = ఎక్కువ టోకెన్లు ప్రాసెస్ చేయబడతాయి
- **రూట్**: ఏ API ఎండ్‌పాయింట్ ఉపయోగించబడిందో నిర్ధారిస్తుంది

**SLM vs LLM ఎప్పుడు ఉపయోగించాలి:**
- **SLM (స్మాల్ లాంగ్వేజ్ మోడల్)**: వేగవంతమైన స్పందనలు, తక్కువ వనరుల వినియోగం, సులభమైన పనులకు మంచిది
- **LLM (లార్జ్ లాంగ్వేజ్ మోడల్)**: అధిక నాణ్యత, మెరుగైన తర్కం, నాణ్యత ముఖ్యమైనప్పుడు ఉపయోగించాలి

**తదుపరి దశలు:**
1. వివిధ ప్రాంప్ట్‌లను ప్రయత్నించి సంక్లిష్టత ఎలా పోలికపై ప్రభావం చూపుతుందో చూడండి
2. ఇతర మోడల్ జంటలతో ప్రయోగాలు చేయండి
3. టాస్క్ సంక్లిష్టత ఆధారంగా తెలివిగా రూట్ చేయడానికి వర్క్‌షాప్ రౌటర్ నమూనాలను (సెషన్ 06) ఉపయోగించండి


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారుల బాధ్యత మేము తీసుకోము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
